In [1]:
#imports and scraping raw data from site
#df is the actual betting line
#df2 has other info, mostly using to get player id to name dict

import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import datetime as dt
import random
import re
from unidecode import unidecode


In [2]:
import urllib3
import json

import urllib
class AppURLopener(urllib.request.FancyURLopener):
    version = "Mozilla/5.0"

    
dog_props_url = 'https://api.underdogfantasy.com/beta/v3/over_under_lines'
opener = AppURLopener()
response = opener.open(dog_props_url)

print(response.status)

data = json.loads(response.read())

C:\Users\Shared.Westover\AppData\Local\Temp\ipykernel_30640\3263823425.py:10: DeprecationWarning: AppURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  opener = AppURLopener()


200


In [3]:
data.keys()

dict_keys(['appearances', 'games', 'over_under_lines', 'players', 'solo_games'])

In [4]:
df = pd.json_normalize(list(data.values()))
df.index = data.keys()
new = df.T['over_under_lines'].values
lines = pd.json_normalize(new)
lines.sample(2)

,id,live_event,live_event_stat,options,over_under_id,rank,stat_value,status,over_under.id,over_under.appearance_stat.id,over_under.appearance_stat.appearance_id,over_under.appearance_stat.display_stat,over_under.appearance_stat.graded_by,over_under.appearance_stat.pickem_stat_id,over_under.appearance_stat.stat,over_under.boost,over_under.scoring_type_id,over_under.title
2247,4d4cacf3-c37f-4105-9e64-d3cfe92e5209,False,None,[{'id': '9c03b3e8-a596-4683-82ac-7289c5b08974'...,1bd8823a-fab7-48af-89eb-c04d25715aa3,199199030,15.5,active,1bd8823a-fab7-48af-89eb-c04d25715aa3,83957e0e-438c-4370-8fee-fa92e9552277,4d5beb7c-ae9e-437a-a516-8a192e91e45f,Fantasy Points,high_score,7d4527ee-5dcf-43b9-8814-8267b3ab0e40,fantasy_points,None,None,K. Tucker + A. Bregman Fantasy Points O/U
2098,6c5bf640-f9b8-4457-8a5d-89dfff39ea4f,False,None,[{'id': 'cbad279b-c2e1-4670-bff1-386d2c85b3dc'...,64675dc9-eb97-4199-b19a-d66c8f8f50e2,182199082,67.5,active,64675dc9-eb97-4199-b19a-d66c8f8f50e2,91ef480e-562f-4ef1-b382-1dfff828fc95,43cbd6be-223c-4659-a598-3922acded092,Points,high_score,61a06e1f-3a67-4d77-96d3-e6b6608ec8bf,points,None,None,Adrian Kempe Points O/U


In [5]:
test =  pd.json_normalize(df.T['players'])
player_df = test.copy(deep=True)
player_df = player_df[['id','first_name', 'last_name',
       'sport_id','team_id']]
player_df = player_df.dropna()#

player_df.loc[player_df.sport_id=='ESPORTS']
x = player_df[player_df.sport_id=='ESPORTS']
p2esport = x[['last_name','first_name']].to_dict('split')
p2esport = dict(p2esport['data'])

In [6]:
test =  pd.json_normalize(df.T['players'])
player_df = test.copy(deep=True)
player_df = player_df[['id','first_name', 'last_name',
       'sport_id','team_id']]
player_df = player_df.dropna()
player_df.loc[player_df['sport_id'] != 'ESPORTS', 'full_name'] = player_df['first_name']+' ' + player_df['last_name']
player_df.loc[player_df['sport_id'] == 'ESPORTS', 'full_name'] = player_df['last_name']
#player_df['full_name'] = player_df['first_name']+' ' + player_df['last_name']
player_df = player_df[['full_name','sport_id','team_id']]
p2l = player_df[['full_name','sport_id']].to_dict('split')
p2tm = player_df[['full_name','team_id']].to_dict('split')
temp = p2l['data']
temp2 = p2tm['data']
p2l = dict(temp)
p2tm = dict(temp2)

In [7]:
player_df

,full_name,sport_id,team_id
0,Trevor Lawrence,NFL,b49b653c-dc7c-516e-b1c2-da1b30d1b1a6
1,Travis Etienne Jr.,NFL,b49b653c-dc7c-516e-b1c2-da1b30d1b1a6
2,Calvin Ridley,NFL,b49b653c-dc7c-516e-b1c2-da1b30d1b1a6
3,Josh Allen,NFL,0719b253-43db-532e-8b5c-6c12c6c3f951
4,James Cook,NFL,0719b253-43db-532e-8b5c-6c12c6c3f951
...,...,...,...
1198,X. Worthy + A. Mitchell,CFBCOMBOS,acafc7a0-f2b7-47ea-b68e-a7bbb4f14b76
1199,A. Anthony + J. Farooq,CFBCOMBOS,059969e5-8906-4018-b150-fc4e94a6ea06
1200,B. Bowers + L. McConkey,CFBCOMBOS,4ee8dc63-186b-4525-8dc4-9fa33bb72ce1
1201,D. Key + T. Robinson,CFBCOMBOS,0af0cd7e-207a-4153-b0d6-9ce488422f8a


In [8]:
player_df.sport_id.unique()

array(['NFL', 'CFB', 'MLB', 'WNBA', 'FIFA', 'ESPORTS', 'RACING',
       'BASKETBALL', 'NHL', 'CFL', 'NBASZN', 'NHLSZN', 'NFLCOMBO',
       'MLBCOMBOS', 'CFBCOMBOS'], dtype=object)

In [9]:
first2 = player_df['full_name'].apply(lambda x: ' '.join(x.split()[0:2]))
names = pd.DataFrame((first2,player_df.full_name))
names = names.T
names.columns=['first2','fullname']
names.loc[names.first2=='Jazz Chisholm']
first2full = names.to_dict('split')
first2full = dict(first2full['data'])

In [10]:
games = pd.json_normalize(df.T['games'].values).dropna(how='all')
#games = games.loc[games['sport_id']=='NBA']
games.tail(3)

,id,away_team_id,away_team_score,home_team_id,home_team_score,match_progress,period,scheduled_at,season_type,sport_id,status,title,type,year
101,52141.0,643ebde6-4db7-48c1-844d-8aa5f8482746,0.0,c3cb471d-2cbd-4cb4-9d57-c8ef7b7d380b,0.0,Fri 10:00pm,0.0,2023-10-07T02:00:00Z,regular,CFL,scheduled,WPG @ BC,Game,2023.0
102,52142.0,828fb713-dc2b-4212-9ad9-5ae1696372b7,0.0,f61e59a6-d7f7-444a-871a-c213320b3d9a,0.0,Sat 07:00pm,0.0,2023-10-07T23:00:00Z,regular,CFL,scheduled,HAM @ SAS,Game,2023.0
103,52143.0,cb31c28e-6b10-459b-a8e8-123f37715b0d,0.0,082be639-ca35-46dd-b8f3-96761e38ac3d,0.0,Mon 01:00pm,0.0,2023-10-09T17:00:00Z,regular,CFL,scheduled,MTL @ OTT,Game,2023.0


In [11]:
def to_central(date_str):
    x = pd.to_datetime(date_str)
    x = x.tz_convert('US/Central')
    return str(x.date())
to_central('2023-05-20T00:40:00Z')

'2023-05-19'

In [12]:
games.scheduled_at = games.scheduled_at.apply(to_central)

In [13]:
#grabbing team names from match title
matchup = games['title']
home = []
away = []
for i in matchup:
    temp = re.split(' @ |, |vs.',i)
    home.append(temp[0])
    away.append(temp[1])
#fixing esports team names
for i in range(len(home)):
    home[i] = home[i].split(': ')[-1]
    
for i in range(len(away)):
    away[i] = away[i].split(': ')[-1]

matchup = pd.DataFrame((away,home,games.away_team_id,games.home_team_id)).T
matchup.columns = ['Home', 'Away','away_id','home_id']
dates = games['scheduled_at'].values
#temp = []
#for i in dates:
#    temp.append(i.split('T')[0])
matchup['Date'] = dates
id2tm =  matchup[['home_id','Home']].to_dict('split')
id2tmaway =  matchup[['away_id','Away']].to_dict('split')
id2tm = dict(id2tm['data'])
id2tmaway = dict(id2tmaway['data'])
id2tm.update(id2tmaway)

In [14]:
games

,id,away_team_id,away_team_score,home_team_id,home_team_score,match_progress,period,scheduled_at,season_type,sport_id,status,title,type,year
0,39372.0,b49b653c-dc7c-516e-b1c2-da1b30d1b1a6,0.0,0719b253-43db-532e-8b5c-6c12c6c3f951,0.0,Sun 09:30am,0.0,2023-10-08,regular,NFL,scheduled,JAX @ BUF,Game,2023.0
1,39376.0,01bfe9d5-f671-57ed-aa60-249fcca9267c,0.0,ecc8eb1b-f714-57a6-bcf3-b183dd6c12a8,0.0,Sun 01:00pm,0.0,2023-10-08,regular,NFL,scheduled,BAL @ PIT,Game,2023.0
2,39377.0,530518ed-91db-57c4-9077-27cc0dd9a293,0.0,ef876de9-81ac-5e60-af6e-0ca1700f3a51,0.0,Sun 01:00pm,0.0,2023-10-08,regular,NFL,scheduled,NO @ NE,Game,2023.0
3,39378.0,d40a4380-49a6-5b5f-ab7d-f5393787ed12,0.0,9153e0a9-da83-5246-ba43-417537f1bcce,0.0,Sun 01:00pm,0.0,2023-10-08,regular,NFL,scheduled,NYG @ MIA,Game,2023.0
4,39373.0,f96aa8db-21c2-5b86-b49d-7e64b4eda61d,0.0,f11f1cf1-9933-5203-8181-95020ee64399,0.0,Sun 01:00pm,0.0,2023-10-08,regular,NFL,scheduled,TEN @ IND,Game,2023.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,50344.0,612c157d-112f-4aa9-a76b-3d17f6b99bb5,0.0,21c32c27-2668-4d04-a209-497292923e36,0.0,Tue 10:34pm,0.0,2023-10-10,regular,NHL,scheduled,SEA @ VGK,Game,2023.0
100,52140.0,32163107-64b6-4522-8f23-df966775492d,0.0,2822c78a-243c-4365-ab0f-bfef88cbac45,0.0,Fri 07:00pm,0.0,2023-10-06,regular,CFL,scheduled,EDM @ TOR,Game,2023.0
101,52141.0,643ebde6-4db7-48c1-844d-8aa5f8482746,0.0,c3cb471d-2cbd-4cb4-9d57-c8ef7b7d380b,0.0,Fri 10:00pm,0.0,2023-10-06,regular,CFL,scheduled,WPG @ BC,Game,2023.0
102,52142.0,828fb713-dc2b-4212-9ad9-5ae1696372b7,0.0,f61e59a6-d7f7-444a-871a-c213320b3d9a,0.0,Sat 07:00pm,0.0,2023-10-07,regular,CFL,scheduled,HAM @ SAS,Game,2023.0


In [15]:
id2tm

{'0719b253-43db-532e-8b5c-6c12c6c3f951': 'BUF',
 'ecc8eb1b-f714-57a6-bcf3-b183dd6c12a8': 'PIT',
 'ef876de9-81ac-5e60-af6e-0ca1700f3a51': 'NE',
 '9153e0a9-da83-5246-ba43-417537f1bcce': 'MIA',
 'f11f1cf1-9933-5203-8181-95020ee64399': 'IND',
 '8699a914-7d44-5a36-b7a6-2063d3ea761f': 'ATL',
 'a8980eb6-327f-5bc5-abf9-de1e944b566d': 'DET',
 'd150534e-6a05-587b-b9e3-50ef86602e20': 'LAR',
 '8459772c-695a-5890-afa1-7ec38da17201': 'ARI',
 '5af1e185-627e-5345-851d-3c65c5b66614': 'MIN',
 '7b3b21be-a209-5dee-a3f7-3fae61f52a1e': 'DEN',
 '7161e62b-de20-56e2-a300-0dc23637faaa': 'SF',
 '5ce78c37-b02c-52da-9e4f-fcd65b6ccb76': 'LV',
 'a2f0fef9-70c3-4b0c-8b8b-fb17fca73e26': 'OKS',
 '9e7a6e95-bea0-4bf8-86f3-9b70b807c704': 'ILL',
 '2503e37c-0856-4d17-8b52-975d1f5d66e9': 'MIZ',
 'a38d0932-4c63-4c04-8351-34565d0045ed': 'TEX',
 '421f09b9-43c3-422a-ac60-7ef2d69f326b': 'WIS',
 '8fa02826-b5c7-494d-a6a3-2fab6cd2ffe2': 'OSU',
 'e58bc430-5aee-4d29-b7cd-6b491c7a41e6': 'MSST',
 '8f1c3220-563e-43fd-b3b8-11fe70dbb149': '

In [16]:
lines = lines[['stat_value','over_under.appearance_stat.stat', 'over_under.title']]
lines
players = lines['over_under.title'].values
guys = []
for i in players:
    temp = i.split()
    final = temp[0] + ' '+temp[1]
    guys.append(final)
    
guys
lines= lines.rename(columns={'stat_value':'Line','over_under.appearance_stat.stat':'Stat', 'over_under.title':'title'})
lines['Player']= guys
lines['Player'] = lines['Player'].apply(lambda x: unidecode(x).replace('_',' '))


In [17]:
def map_name(x):
    if x is None:
        return None
    if x in first2full:
        return first2full[x]
    elif x.split(': ')[-1] in first2full:
        return x.split(': ')[-1]
    elif x.split()[-1] in first2full:
        return x.split()[-1]
    else:
        return x

lines['Player'] = lines['Player'].apply(lambda x: map_name(x))
lines.Player.unique()

array(['Trevor Lawrence', 'Travis Etienne Jr.', 'Calvin Ridley', ...,
       'J. Thrash', 'R. Wilson', 'C. Stover'], dtype=object)

In [18]:
league = []
for p in guys:
    if p in p2l:
        league.append(p2l[p])
    elif p.split()[-1] in p2l:
        league.append(p2l[p.split()[-1]])
    else:
        league.append('other')
league[0:5]

['NFL', 'other', 'NFL', 'NFL', 'NFL']

In [19]:
guys[1000:1007]

['Savion Williams',
 'JP Richardson',
 'Dylan Wright',
 'JP Richardson',
 'Jared Wiley',
 'Rocco Becht',
 'Rocco Becht']

In [20]:
team_id = []
for p in guys:
    if p in p2l:
        team_id.append(p2tm[p])
    elif p.split()[-1] in p2l:
        team_id.append(p2tm[p.split()[-1]])
    else:
        team_id.append('other')
team_id[0:5]

['b49b653c-dc7c-516e-b1c2-da1b30d1b1a6',
 'other',
 'b49b653c-dc7c-516e-b1c2-da1b30d1b1a6',
 'b49b653c-dc7c-516e-b1c2-da1b30d1b1a6',
 '0719b253-43db-532e-8b5c-6c12c6c3f951']

In [21]:
lines['League'] = league
lines

,Line,Stat,title,Player,League
0,17.55,fantasy_points,Trevor Lawrence Fantasy Points O/U,Trevor Lawrence,NFL
1,11.85,fantasy_points,Travis Etienne Jr. Fantasy Points O/U,Travis Etienne Jr.,other
2,13.5,receiving_yac,Calvin Ridley Yards After Catch O/U,Calvin Ridley,NFL
3,10.05,fantasy_points,Calvin Ridley Fantasy Points O/U,Calvin Ridley,NFL
4,261.5,passing_yds,Josh Allen Passing Yards O/U,Josh Allen,NFL
...,...,...,...,...,...
2326,133.5,receiving_yds,A. Armstrong + J. Watkins Receiving Yards O/U,A. Armstrong,other
2327,121.5,receiving_yds,C. Harris + D. Wade Receiving Yards O/U,C. Harris,other
2328,128.5,receiving_yds,J. Thrash + M. Evans Receiving Yards O/U,J. Thrash,other
2329,101.5,receiving_yds,R. Wilson + D. Jackson Receiving Yards O/U,R. Wilson,other


In [22]:
def map_team(x):
    if x in p2tm:
        return p2tm[x]
    elif p.split()[-1] in p2l:
        return p2tm[p.split()[-1]]
    else:
        return x

lines['team_id'] = lines.Player.apply(lambda x: map_team(x))
lines

,Line,Stat,title,Player,League,team_id
0,17.55,fantasy_points,Trevor Lawrence Fantasy Points O/U,Trevor Lawrence,NFL,b49b653c-dc7c-516e-b1c2-da1b30d1b1a6
1,11.85,fantasy_points,Travis Etienne Jr. Fantasy Points O/U,Travis Etienne Jr.,other,b49b653c-dc7c-516e-b1c2-da1b30d1b1a6
2,13.5,receiving_yac,Calvin Ridley Yards After Catch O/U,Calvin Ridley,NFL,b49b653c-dc7c-516e-b1c2-da1b30d1b1a6
3,10.05,fantasy_points,Calvin Ridley Fantasy Points O/U,Calvin Ridley,NFL,b49b653c-dc7c-516e-b1c2-da1b30d1b1a6
4,261.5,passing_yds,Josh Allen Passing Yards O/U,Josh Allen,NFL,0719b253-43db-532e-8b5c-6c12c6c3f951
...,...,...,...,...,...,...
2326,133.5,receiving_yds,A. Armstrong + J. Watkins Receiving Yards O/U,A. Armstrong,other,A. Armstrong
2327,121.5,receiving_yds,C. Harris + D. Wade Receiving Yards O/U,C. Harris,other,C. Harris
2328,128.5,receiving_yds,J. Thrash + M. Evans Receiving Yards O/U,J. Thrash,other,J. Thrash
2329,101.5,receiving_yds,R. Wilson + D. Jackson Receiving Yards O/U,R. Wilson,other,R. Wilson


In [23]:
def map_id(x):
    if x in id2tm:
        return id2tm[x]
    else:
        return x
#lines['team_id'] = lines.team_id.apply(lambda x: map_id(x))


In [24]:
final = pd.merge(lines,matchup, how='left',left_on='team_id',right_on='away_id')
final = final.drop(['Home', 'Away','home_id','away_id'], axis=1)
final2 = pd.merge(final,matchup, how='left',left_on='team_id',right_on='home_id')
final2 = final2.drop(['Home', 'Away','home_id','away_id'], axis=1)
final2['Date'] = final2['Date_x'].fillna(final2['Date_y'])
final2 = final2.drop(['Date_x', 'Date_y'], axis=1)
final2['team_id'] = final2.team_id.apply(lambda x: map_id(x))
final2#.loc[final.Player=='Jimmy Butler']

,Line,Stat,title,Player,League,team_id,Date
0,17.55,fantasy_points,Trevor Lawrence Fantasy Points O/U,Trevor Lawrence,NFL,JAX,2023-10-08
1,11.85,fantasy_points,Travis Etienne Jr. Fantasy Points O/U,Travis Etienne Jr.,other,JAX,2023-10-08
2,13.5,receiving_yac,Calvin Ridley Yards After Catch O/U,Calvin Ridley,NFL,JAX,2023-10-08
3,10.05,fantasy_points,Calvin Ridley Fantasy Points O/U,Calvin Ridley,NFL,JAX,2023-10-08
4,261.5,passing_yds,Josh Allen Passing Yards O/U,Josh Allen,NFL,BUF,2023-10-08
...,...,...,...,...,...,...,...
2326,133.5,receiving_yds,A. Armstrong + J. Watkins Receiving Yards O/U,A. Armstrong,other,A. Armstrong,NaN
2327,121.5,receiving_yds,C. Harris + D. Wade Receiving Yards O/U,C. Harris,other,C. Harris,NaN
2328,128.5,receiving_yds,J. Thrash + M. Evans Receiving Yards O/U,J. Thrash,other,J. Thrash,NaN
2329,101.5,receiving_yds,R. Wilson + D. Jackson Receiving Yards O/U,R. Wilson,other,R. Wilson,NaN


In [25]:
final2.columns = ['Line', 'Stat', 'title', 'Player', 'League', 'Team', 'Date']

In [26]:
df = final2[['Line', 'Stat', 'Player', 'League', 'Team', 'Date']].copy(deep=True)
df

,Line,Stat,Player,League,Team,Date
0,17.55,fantasy_points,Trevor Lawrence,NFL,JAX,2023-10-08
1,11.85,fantasy_points,Travis Etienne Jr.,other,JAX,2023-10-08
2,13.5,receiving_yac,Calvin Ridley,NFL,JAX,2023-10-08
3,10.05,fantasy_points,Calvin Ridley,NFL,JAX,2023-10-08
4,261.5,passing_yds,Josh Allen,NFL,BUF,2023-10-08
...,...,...,...,...,...,...
2326,133.5,receiving_yds,A. Armstrong,other,A. Armstrong,NaN
2327,121.5,receiving_yds,C. Harris,other,C. Harris,NaN
2328,128.5,receiving_yds,J. Thrash,other,J. Thrash,NaN
2329,101.5,receiving_yds,R. Wilson,other,R. Wilson,NaN


In [27]:
df = df.reset_index(drop=True)

In [28]:
df.Stat.unique()

array(['fantasy_points', 'receiving_yac', 'passing_yds', 'rushing_yds',
       'rushing_att', 'passing_att', 'passing_comps', 'passing_ints',
       'passing_and_rushing_yds', 'receiving_rec', 'receiving_yds',
       'rush_rec_yds', 'first_downs', 'receiving_tgts',
       'extra_points_made', 'kicking_points', 'assists',
       'tackles_and_assists', 'tackles', 'field_goals_made',
       'passing_tds', 'rush_rec_tds', 'sacks', 'rushing_tds',
       'strikeouts', 'runs_allowed', 'hits_allowed', 'walks_allowed',
       'hits_runs_rbis', 'batter_strikeouts', 'singles', 'total_bases',
       'pitch_outs', 'runs', 'walks', 'points', 'rebounds', 'saves',
       'goals_against', 'shots_attempted', 'passes', 'goals_assists',
       'chances_created', 'shots_on_target', 'fouls_against', 'aces',
       'double_faults', 'sets_lost', 'breakpoints_won',
       'service_games_lost', 'games_won', 'games_lost', 'games_played',
       'strokes', 'birdies_or_better', 'bogeys_or_worse',
       'cumulativ

In [29]:
syntax = {
    'pts_rebs_asts': 'PTS+TRB+AST',
    'pts_rebs': 'PTS+TRB',
    'points': 'PTS',
    'free_throws_made': 'FT',
    'fantasy_points': 'Fantasy',
    'rebounds': 'TRB',
    'assists': 'AST',
    'steals': 'STL',
    'blocks': 'BLK',
    'blks_stls': 'BLK+STL',
    'turnovers': 'TO',
    'rebs_asts': 'TRB+AST',
    'pts_asts': 'PTS+AST',
    'three_points_made': '3PM',
    'shots': 'Shots',
    'goals': 'Goals',
    'saves': 'Goalie Saves',
    'clearances': 'Clearances',
    'goals_and_assists': 'Goals and Assists',
    'tackles': 'Tackles',
    'interceptions': 'Interceptions',
    'crosses': 'Crosses',
    'goals_against': 'Goal Against',
    'game_1_and_2_kills': 'MAPS 1-2 Kills',
    'map_1_and_2_kills': 'MAPS 1-2 Kills',
    'Headshots on Maps 1+2': 'MAPS 1-2 Headshots',
    'strokes': 'Strokes',
    'bogeys_or_worse': 'Bogeys or Worse',
    'birdies_or_better': 'Birdies or Better',
    'aces': 'Aces',
    'breakpoints_won': 'Break Points Won',
    'games_won': 'Games Won',
    'games_lost': 'Games Lost',
    'total_games': 'Total Games',
    'double_faults': 'Double Faults',
    'service_games_lost': 'Service Games Lost',
    'Fastest Laps': 'Fastest Laps',
    'position': 'position',
    'passing_yds': 'Pass Yards',
    'rushing_yds': 'Rush Yards',
    'receiving_yds': 'Receiving Yards' ,
    'fantasy_points': 'Hitter Fantasy Score',
    'strikeouts': 'Pitcher Strikeouts',
    'hits_runs_rbis': 'PTS',
    'singles':'Singles',
    'total_bases': 'Total Bases',
    'walks': 'Walks Allowed',
    'hits': 'Hits Allowed',
                }

In [30]:
df = df.replace(syntax)

In [31]:
df.loc[df['League'] == 'ESPORTS']

,Line,Stat,Player,League,Team,Date
1570,36.5,MAPS 1-2 Kills,amy,ESPORTS,Acend Rising,2023-10-06
1571,33.0,MAPS 1-2 Kills,Joliinaa,ESPORTS,Acend Rising,2023-10-06
1572,30.0,MAPS 1-2 Kills,Val: schnellAE,ESPORTS,Val: schnellAE,NaN
1573,24.5,MAPS 1-2 Kills,speedowka,ESPORTS,Acend Rising,2023-10-06
1574,23.0,MAPS 1-2 Kills,h6tedthemost,ESPORTS,Acend Rising,2023-10-06
...,...,...,...,...,...,...
1852,11.0,headshots_on_maps_1_2,Cooper,ESPORTS,Mythic,2023-10-06
1853,9.5,headshots_on_maps_1_2,h4rn,ESPORTS,Pompa,2023-10-06
1854,7.5,MAPS 1-2 Kills,MetroArcher,ESPORTS,Bytewave,2023-10-06
2260,11.5,Hitter Fantasy Score,J. Martinez + J. Outman,ESPORTS,3566eb7e-69fd-4f9f-a5ea-6f8db014b027,NaN


In [32]:
df.loc[df.Player=='Krimbo']

,Line,Stat,Player,League,Team,Date


In [33]:
today = dt.date.today()

In [34]:
str(today)

'2023-10-06'

In [35]:
df.to_csv(f'Lines/doglines{today.month}_{today.day}')

In [36]:
df.loc[df.League=='ESPORTS']

,Line,Stat,Player,League,Team,Date
1570,36.5,MAPS 1-2 Kills,amy,ESPORTS,Acend Rising,2023-10-06
1571,33.0,MAPS 1-2 Kills,Joliinaa,ESPORTS,Acend Rising,2023-10-06
1572,30.0,MAPS 1-2 Kills,Val: schnellAE,ESPORTS,Val: schnellAE,NaN
1573,24.5,MAPS 1-2 Kills,speedowka,ESPORTS,Acend Rising,2023-10-06
1574,23.0,MAPS 1-2 Kills,h6tedthemost,ESPORTS,Acend Rising,2023-10-06
...,...,...,...,...,...,...
1852,11.0,headshots_on_maps_1_2,Cooper,ESPORTS,Mythic,2023-10-06
1853,9.5,headshots_on_maps_1_2,h4rn,ESPORTS,Pompa,2023-10-06
1854,7.5,MAPS 1-2 Kills,MetroArcher,ESPORTS,Bytewave,2023-10-06
2260,11.5,Hitter Fantasy Score,J. Martinez + J. Outman,ESPORTS,3566eb7e-69fd-4f9f-a5ea-6f8db014b027,NaN
